In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import numpy as np
import requests
from scipy.stats import poisson
from difflib import get_close_matches
from pulp import *
import time
from bs4 import BeautifulSoup
import json

In [3]:
def fbref_url(): 

    url = 'https://fbref.com/en/comps/9/Premier-League-Stats'
    r = requests.get(url)
    html_content = r.text
    soup = BeautifulSoup(html_content, 'lxml')
    htmltable = soup.find('div', { 'id' : 'all_results2022-202391' })
    list_squad_url = {a.get('href') for a in htmltable.find_all('a', href=True) if 'squads' in a.get('href')}
    list_squad_url = ['https://fbref.com' + url for url in list_squad_url]
    with open("squad_url.json", 'w') as f:
        json.dump(list_squad_url, f, indent=2) 

    list_player_url = []
    for url in list_squad_url:
        print(url)
        r = requests.get(url)
        html_content = r.text
        soup = BeautifulSoup(html_content, 'lxml')
        htmltable = soup.find('div', { 'id' : 'all_stats_standard' })
        list_player_url += [a.get('href') for a in htmltable.find_all('a', href=True) if 'matchlogs' in a.get('href')]
        time.sleep(5)
    list_player_url = ['https://fbref.com' + url for url in list_player_url]
    with open("player_url.json", 'w') as f:
        json.dump(list_player_url, f, indent=2) 

In [4]:
def fbref_parser():
  """
  Parse player's match logs data based on player url.

  Parameters: metrics, EWMA alpha
  """
  metrics = ['npxG', 'xA']
  alpha = 0.4

  # obtaining player_url
  with open("player_url.json", 'r') as f:
    list_player_url = json.load(f)

  # parsing data from fbref
  small_dfs = []
  for i, match_url in enumerate(list_player_url):
    time.sleep(2.5)
    if i % 50 == 0:
      print(f'{i}/{len(list_player_url)} completed.')

    try:
      r = requests.get(match_url)
      df_raw = pd.read_html(r.text)[0]
      df_raw.columns = df_raw.columns.droplevel()
      df = df_raw[(df_raw['Comp']=='Premier League') & ((df_raw['Start']=='Y') | (df_raw['Start']=='Y*'))]
      if len(df) == 0:
        continue
      df = df[metrics]
      df.astype(float)  

      df = df.ewm(alpha=alpha, adjust=False).mean().tail(1).reset_index()
      df['Name'] = match_url.split('/')[-1][:-11].replace('-', ' ')
      df['team'] = df_raw[df_raw['Comp']=='Premier League']['Squad'].tail(1).values
      df.columns = df.columns.str.lower()
      small_dfs.append(df)

    except Exception as e: 
      print(e)
      print(match_url)

  df_final = pd.concat(small_dfs, ignore_index=True)
  df_final = df_final[['name', 'team', 'npxg', 'xa']]

  team_dict = {'Leeds United': 'Leeds',
              'Leicester City': 'Leicester',
              'Manchester City': 'Man City',
              'Manchester Utd': 'Man Utd',
              'Newcastle Utd': 'Newcastle',
              "Nott'ham Forest": "Nott'm Forest",
              'Tottenham': 'Spurs'}
  df_final['team'] = df_final['team'].replace(team_dict)
  df_final.fillna(0, inplace=True)
  df_player_xg = df_final

  return df_player_xg

In [122]:
df_final = fbref_parser()

0/359 completed.
20/359 completed.
40/359 completed.
60/359 completed.
No tables found
https://fbref.com/en/players/6a713852/matchlogs/2022-2023/summary/Robert-Sanchez-Match-Logs
80/359 completed.
100/359 completed.
120/359 completed.
140/359 completed.
160/359 completed.
180/359 completed.
200/359 completed.
220/359 completed.
240/359 completed.
260/359 completed.
280/359 completed.
300/359 completed.
320/359 completed.
340/359 completed.


In [124]:
def squad_parser():

    url = 'https://fbref.com/en/comps/9/Premier-League-Stats'
    r = requests.get(url)
    df = pd.read_html(r.text)

    columns = {
        'Unnamed: 0_level_0_Squad': 'squad',
        'Per 90 Minutes_xG': 'npxg',
        'Per 90 Minutes_xA': 'xa'
    }
    squad_xg_df = df[2]
    squad_xg_df.columns = ["_".join(a) for a in squad_xg_df.columns.to_flat_index()]
    squad_xg_df.rename(columns=columns, inplace=True)
    squad_xg_df = squad_xg_df[['squad', 'npxg', 'xa']]
    squad_xg_df['npxg_boost'] = squad_xg_df['npxg'].apply(lambda x: x / squad_xg_df['npxg'].mean()).round(2)
    squad_xg_df['xa_boost'] = squad_xg_df['xa'].apply(lambda x: x / squad_xg_df['xa'].mean()).round(2)
    

    squad_xcs_df = df[3]
    squad_xcs_df.columns = ["_".join(a) for a in squad_xcs_df.columns.to_flat_index()]
    squad_xcs_df.rename(columns=columns, inplace=True)
    squad_xcs_df = squad_xcs_df[['squad', 'npxg', 'xa']]
    squad_xcs_df['npxg_boost'] = squad_xcs_df['npxg'].apply(lambda x: x / squad_xcs_df['npxg'].mean()).round(2)
    squad_xcs_df['xa_boost'] = squad_xcs_df['xa'].apply(lambda x: x / squad_xcs_df['xa'].mean()).round(2)
    
    return squad_xg_df, squad_xcs_df

In [5]:
def team_xg(week, alpha=0.3):

    url = 'https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures'
    r = requests.get(url)
    df = pd.read_html(r.text)[0]

    team_xg_df1 = df[['Wk', 'Home', 'xG']].rename(columns={'Home':'Squad'})
    team_xg_df2 = df[['Wk', 'Away', 'xG.1']].rename(columns={'Away':'Squad', 'xG.1':'xG'})
    team_xg_df = pd.concat([team_xg_df1, team_xg_df2], axis=0, ignore_index=True)
    team_xg_df = team_xg_df[team_xg_df['Wk']<week].sort_values('Wk')

    team_xg_dict = dict()
    for team in team_xg_df['Squad'].unique():
        team_xg_dict[team] = team_xg_df[team_xg_df['Squad']==team]['xG'].ewm(alpha=alpha, adjust=False).mean().tail(1).values[0].round(3)
    team_xg_df = pd.DataFrame.from_dict(team_xg_dict, orient='index').reset_index().rename(columns={'index':'squad', 0: 'xg'})
    team_xg_df['xg_boost'] = team_xg_df['xg'].apply(lambda x: x / team_xg_df['xg'].mean()).round(2)
    team_xg_df.sort_values('xg', ascending=False, inplace=True)

    # Computing team xgc
    team_xgc_df1 = df[['Wk', 'Home', 'xG.1']].rename(columns={'Home':'Squad','xG.1':'xG'})
    team_xgc_df2 = df[['Wk', 'Away', 'xG']].rename(columns={'Away':'Squad'})
    team_xgc_df = pd.concat([team_xgc_df1, team_xgc_df2], axis=0, ignore_index=True)
    team_xgc_df = team_xgc_df[team_xgc_df['Wk']<week].sort_values('Wk')

    team_xgc_dict = dict()
    for team in team_xgc_df['Squad'].unique():
        team_xgc_dict[team] = team_xgc_df[team_xgc_df['Squad']==team]['xG'].ewm(alpha=alpha, adjust=False).mean().tail(1).values[0].round(3)
    team_xgc_df = pd.DataFrame.from_dict(team_xgc_dict, orient='index').reset_index().rename(columns={'index':'squad', 0: 'xgc'})
    team_xgc_df['xgc_boost'] = team_xgc_df['xgc'].apply(lambda x: x / team_xgc_df['xgc'].mean()).round(2)
    team_xgc_df.sort_values('xgc', ascending=False, inplace=True)

    return team_xg_df, team_xgc_df

team_xg_df, team_xgc_df = team_xg(week=6)
team_xg_df


,squad,xg,xg_boost
10,Manchester City,2.749,2.11
1,Arsenal,2.086,1.60
2,Liverpool,2.016,1.55
9,Brighton,1.631,1.25
8,Brentford,1.504,1.15
18,Tottenham,1.471,1.13
11,Newcastle Utd,1.425,1.09
16,Fulham,1.390,1.07
7,Chelsea,1.390,1.07
17,Leeds United,1.383,1.06


In [7]:
def fixture_xg(week):
    
    def get_fixtures(week):
        url = 'https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures'
        r = requests.get(url)
        df = pd.read_html(r.text)[0]
        fixtures_df = df[['Wk', 'Home', 'Away']]
        fixtures_df = pd.concat([fixtures_df, fixtures_df.rename(columns={'Home':'Away', 'Away':'Home'})])
        fixtures_df = fixtures_df.rename(columns={'Home':'squad', 'Away':'opp'})
        return fixtures_df[fixtures_df['Wk']==week]

    fixtures = get_fixtures(week)

    df_xg_full = fixtures.merge(team_xgc_df, on='squad', how='left').merge(team_xg_df, left_on='opp', right_on='squad', how='left')
    df_xg_full['xgc_true'] = df_xg_full['xgc'] * df_xg_full['xg_boost']
    team_dict = {'Leeds United': 'Leeds',
            'Leicester City': 'Leicester',
            'Manchester City': 'Man City',
            'Manchester Utd': 'Man Utd',
            'Newcastle Utd': 'Newcastle',
            "Nott'ham Forest": "Nott'm Forest",
            'Tottenham': 'Spurs'}
    df_xgc = df_xg_full[['squad_x', 'xgc_true']].rename(columns={'squad_x': 'team', 'xgc_true': 'xGC'})
    df_xgc['team'] = df_xgc['team'].replace(team_dict)
    df_xg = df_xg_full[['opp', 'xgc_boost']].rename(columns={'opp': 'team'})
    df_xg['team'] = df_xg['team'].replace(team_dict)

    return df_xg, df_xgc

# df_xg, df_xgc = fixture_xg()

In [239]:
url = 'https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures'
r = requests.get(url)
df = pd.read_html(r.text)[0]
fixtures_df = df[['Wk', 'Home', 'Away']]
test = pd.concat([fixtures_df, fixtures_df.rename(columns={'Home':'Away', 'Away':'Home'})])
test = test.rename(columns={'Home':'squad', 'Away':'opp'})
wk5 = test[test['Wk']==6]

df_xg_full = wk5.merge(team_xgc_df, on='squad', how='left').merge(team_xg_df, left_on='opp', right_on='squad', how='left')
df_xg_full['xgc_true'] = df_xg_full['xgc'] * df_xg_full['xg_boost']
df_xg = df_xg_full[['squad_x', 'xgc_true']].rename(columns={'squad_x': 'team', 'xgc_true': 'xGC'})
df_xg1 = df_xg_full[['opp', 'xgc_boost']].rename(columns={'opp': 'team'})

team_dict = {'Leeds United': 'Leeds',
            'Leicester City': 'Leicester',
            'Manchester City': 'Man City',
            'Manchester Utd': 'Man Utd',
            'Newcastle Utd': 'Newcastle',
            "Nott'ham Forest": "Nott'm Forest",
            'Tottenham': 'Spurs'}
df_xg1['team'] = df_xg1['team'].replace(team_dict)
df_xg1.sort_values('xgc_boost', ascending=False)
df_xg

,team,xGC
0,Everton,2.68576
1,Brentford,1.18826
2,Nott'ham Forest,0.81778
3,Chelsea,0.69930
4,Tottenham,0.97732
5,Newcastle Utd,1.05618
6,Wolves,0.73002
7,Aston Villa,3.38940
8,Brighton,0.82236
9,Manchester Utd,2.55672


In [172]:
df_player_xg = df_final
df_player_xg

,name,team,npxg,xa,key
0,Lucas Digne,Aston Villa,0.0480,0.1496,Lucas Digne
1,Emiliano Martinez,Aston Villa,0.0000,0.0000,Emiliano Marcondes
2,Matty Cash,Aston Villa,0.0400,0.0480,Matty Cash
3,Boubacar Kamara,Aston Villa,0.0000,0.0000,Boubacar Kamara
4,John McGinn,Aston Villa,0.0216,0.0288,John McGinn
...,...,...,...,...,...
278,Marc Cucurella,Chelsea,0.0600,0.1000,Marc Cucurella
279,NGolo Kante,Chelsea,0.0600,0.1200,N'Golo Kanté
280,Cesar Azpilicueta,Chelsea,0.0000,0.1000,César Azpilicueta
281,Ben Chilwell,Chelsea,0.0000,0.0000,Ben Chilwell


In [8]:
def fpl_parser():

    print('Parsing FPL data...')
    # bootstrap-static
    url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
    r = requests.get(url)
    json = r.json()

    elements_df = pd.DataFrame(json['elements'])
    elements_types_df = pd.DataFrame(json['element_types'])
    teams_df = pd.DataFrame(json['teams'])

    # data prepocessing
    elements_df['name'] = elements_df['first_name'] + ' ' + elements_df['second_name']
    elements_df['name_alt'] = elements_df['name'].apply(lambda x: f'{x.split()[0]} {x.split()[-1]}')
    elements_df['position'] = elements_df['element_type'].map({1:'GK',
                                                                2:'DEF',
                                                                3:'MID',
                                                                4:'FWD'})
    elements_df['team'] = elements_df['team'].map(teams_df.set_index('id')['name'])
    elements_df['now_cost'] = elements_df['now_cost']/10

    # computing game minutes 3MA
    min_ma_dict = dict()
    for ele in elements_df['id']:
        try:
            url = f'https://fantasy.premierleague.com/api/element-summary/{ele}/'
            r = requests.get(url)
            json = r.json()

            min_ma = pd.DataFrame(json['history']).tail(3)['minutes'].mean()
            min_ma_dict[ele] = round(min_ma, 0)

        except:
            print(f'Skipping player {ele} due to error encountered')

    elements_df['mins_ma'] = elements_df['id'].map(min_ma_dict)

    columns_add_list = ['name', 'team', 'position', 
                        'now_cost', 'form', 'ict_index', 'selected_by_percent', 'minutes',
                        'mins_ma', 'transfers_in','total_points', 'name_alt']
    final_df = elements_df.copy()[columns_add_list]
    # final_df['is_dgw'] = final_df['team'].isin(['Aston Villa', 'Leeds', 'Newcastle', 'Southampton', 'Watford', 'Wolves'])
   
    return final_df

In [159]:
final_df = fpl_parser()

Parsing FPL data...


In [185]:
df_fpl = final_df
df_fpl

,name,team,position,now_cost,form,ict_index,selected_by_percent,minutes,mins_ma,transfers_in,total_points,name_alt
0,Cédric Alves Soares,Arsenal,DEF,4.3,0.0,0.0,0.2,0,0.0,3082,0,Cédric Soares
1,Granit Xhaka,Arsenal,MID,5.0,5.5,25.3,2.8,357,89.0,252632,22,Granit Xhaka
2,Mohamed Elneny,Arsenal,MID,4.4,0.5,0.8,1.2,90,30.0,86040,2,Mohamed Elneny
3,Rob Holding,Arsenal,DEF,4.3,0.2,0.0,0.1,1,0.0,1968,1,Rob Holding
4,Thomas Partey,Arsenal,MID,5.0,2.0,11.2,0.5,270,60.0,25215,8,Thomas Partey
...,...,...,...,...,...,...,...,...,...,...,...,...
596,Joseph Hodge,Wolves,MID,4.5,0.0,0.0,0.0,0,0.0,2586,0,Joseph Hodge
597,Connor Ronan,Wolves,MID,4.5,0.0,0.0,0.1,0,0.0,5796,0,Connor Ronan
598,Chem Campbell,Wolves,MID,4.4,0.2,0.0,0.3,5,0.0,22279,1,Chem Campbell
599,Gonçalo Manuel Ganchinho Guedes,Wolves,MID,6.0,3.3,13.4,0.4,194,65.0,43245,10,Gonçalo Guedes


In [9]:
def xgc_parser():

  # df_xg = pd.read_csv('/Users/user/Downloads/team_xg_gw38.csv', sep='\t')
  # df1 = df_xg[['Home Team', 'xG Away']].rename(columns={'Home Team': 'team', 'xG Away': 'xGC'})
  # df2 = df_xg[['Away Team', 'xG Home']].rename(columns={'Away Team': 'team', 'xG Home': 'xGC'})
  # df_xg = pd.concat([df1, df2], axis=0, ignore_index=True)

  # team_dict = {
  #   'Man United': 'Man Utd',
  #   'Tottenham': 'Spurs'
  # }
  # df_xg['team'] = df_xg['team'].replace(team_dict)

  def def_ev(xgc):
    ev_cs = poisson.pmf(0, xgc) * 4
    ev_concede = -sum(np.fromiter((poisson.pmf(k, xgc)*(k//2) for k in range(4)), float)) 
    
    return ev_cs + ev_concede

  def mid_ev(xgc):
    ev_cs = poisson.pmf(0, xgc) * 1
    
    return ev_cs 

  df_xgc['ev_def'] = df_xgc['xGC'].apply(def_ev)
  df_xgc['ev_mid'] = df_xgc['xGC'].apply(mid_ev)

  team_dict = {'Leeds United': 'Leeds',
            'Leicester City': 'Leicester',
            'Manchester City': 'Man City',
            'Manchester Utd': 'Man Utd',
            'Newcastle Utd': 'Newcastle',
            "Nott'ham Forest": "Nott'm Forest",
            'Tottenham': 'Spurs'}
  df_xgc['team'] = df_xgc['team'].replace(team_dict)  

  return df_xgc.sort_values('xGC', ascending=False)

In [245]:
dff = xg_parser()
dff

,team,xGC,ev_def,ev_mid
7,Aston Villa,3.38940,-0.277711,0.033729
0,Everton,2.68576,-0.193296,0.068169
9,Man Utd,2.55672,-0.159296,0.077559
14,Fulham,1.97456,0.106540,0.138822
15,Crystal Palace,1.64154,0.370986,0.193682
18,Leicester,1.57604,0.435421,0.206792
11,Leeds,1.36505,0.675292,0.255368
13,West Ham,1.34302,0.703393,0.261056
1,Brentford,1.18826,0.918638,0.304751
5,Newcastle,1.05618,1.128857,0.347782


In [5]:
def calc_gw():
    
    TEAM_LIST = ['Arsenal','Aston Villa','Brentford','Brighton','Burnley','Chelsea',
    'Crystal Palace','Everton','Leeds','Leicester','Liverpool','Man City',
    'Man Utd','Newcastle','Norwich','Southampton','Spurs','Watford','West Ham','Wolves']

    bgw_list = list(set(TEAM_LIST) - set(df_xg['team'].unique()))
    dgw_list = df_xg[df_xg.duplicated(['team'])]['team'].tolist()

    return dgw_list, bgw_list

In [10]:
def merge_df():

    # merge df_fbref and df_fpl
    # def name_parser(x):
    #     try:
    #         return get_close_matches(x, df_fpl['name_alt'], n=1)[0]
    #     except:
    #         return x

    def name_parser(df):
        try:
            return get_close_matches(df['name'], df_fpl[df_fpl['team']==df['team']]['name_alt'], n=1)[0]
        except:
            return df['name']

    # df_player_xg['key'] = df_player_xg['name'].apply(name_parser)
    df_player_xg['key'] = df_player_xg.apply(name_parser, axis=1)
    # df_player_xg['key'].replace(name_dict, inplace=True)

    df_final = df_player_xg.merge(df_fpl, how='left', left_on='key', right_on='name_alt')
    col_to_drop = ['key', 'name_y', 'team_x']
    df_final.drop(col_to_drop, axis=1, inplace=True)
    df_final.rename(columns={'name_x': 'name', 'team_y': 'team'}, inplace=True)

    # merge with df_xg
    df_xg_sum = df_xgc.groupby('team')[['ev_def', 'ev_mid']].sum() # sum scores for dgw
    df_final = df_final.merge(df_xg_sum, how='left', on='team')

    df_final = df_final.merge(df_xg, on='team', how='left')
    df_final['xg_true'] = df_final['npxg'] * df_final['xgc_boost']
    df_final['xa_true'] = df_final['xa'] * df_final['xgc_boost']

    # remove players with 0 form
    df_final['form'] = df_final['form'].astype('float')
    df_final = df_final[df_final['mins_ma']>60]
    
    return df_final
    

In [253]:
df_final = merge_df()

In [11]:
def compute_ev():

  def compute_score(df):

    # computing score
    # if df['team'] in bgw_list:
    #   return 0

    if df['position'] == 'FWD':
      score_goals = sum(np.fromiter(((poisson.pmf(i, df['xg_true'])*4*i for i in range(4))), float))
      score_assists = sum(np.fromiter(((poisson.pmf(i, df['xa_true'])*3*i for i in range(4))), float))
      # if df['team'] in dgw_list:
      #   ev_score = (2 + score_goals + score_assists) * 2
      # else:
      ev_score = 2 + score_goals + score_assists

    if df['position'] == 'MID':
      score_goals = sum(np.fromiter(((poisson.pmf(i, df['xg_true'])*5*i for i in range(4))), float))
      score_assists = sum(np.fromiter(((poisson.pmf(i, df['xa_true'])*3*i for i in range(4))), float))
      # if df['team'] in dgw_list:
      #   ev_score = 2 * (2 + score_goals + score_assists) + df['ev_mid']
      # else:
      ev_score = 2 + score_goals + score_assists + df['ev_mid']

    if df['position']== 'DEF':
      score_goals = sum(np.fromiter(((poisson.pmf(i, df['xg_true'])*6*i for i in range(4))), float))
      score_assists = sum(np.fromiter(((poisson.pmf(i, df['xa_true'])*4*i for i in range(4))), float))
      # if df['team'] in dgw_list:
      #   ev_score = 2 * (2 + score_goals + score_assists) + df['ev_def']
      # else:
      ev_score = 2 + score_goals + score_assists + df['ev_def']

    if df['position'] == 'GK':
      ev_score = df['ev_def']

    return ev_score

  df_final['proj_score'] = round(df_final.apply(compute_score, axis=1), 2)
  print('Dropping these rows due to missing data...')
  print(df_final[df_final['proj_score'].isna()])
  df_final.dropna(subset=['proj_score'], inplace=True)

  return df_final
    

In [256]:
df_finall = compute_ev()

Dropping these rows due to missing data...
Empty DataFrame
Columns: [name, npxg, xa, team, position, now_cost, form, ict_index, selected_by_percent, minutes, mins_ma, transfers_in, total_points, name_alt, ev_def, ev_mid, xgc_boost, xg_true, xa_true, proj_score]
Index: []


In [260]:
df_finall.sort_values('proj_score', ascending=False)

,name,npxg,xa,team,position,now_cost,form,ict_index,selected_by_percent,minutes,mins_ma,transfers_in,total_points,name_alt,ev_def,ev_mid,xgc_boost,xg_true,xa_true,proj_score
110,Mohamed Salah,0.7048,0.2368,Liverpool,MID,13.0,6.2,47.0,56.0,360.0,90.0,482586.0,25.0,Mohamed Salah,1.403688,0.404785,1.32,0.930336,0.312576,7.67
219,Bernardo Silva,0.4400,0.3800,Man City,MID,6.8,6.0,30.5,7.2,217.0,68.0,334279.0,24.0,Bernardo Silva,3.035453,0.766255,1.22,0.536800,0.463600,6.78
213,Erling Haaland,1.0848,0.0672,Man City,FWD,11.8,10.2,51.4,66.7,323.0,82.0,2202845.0,41.0,Erling Haaland,3.035453,0.766255,1.22,1.323456,0.081984,6.75
273,Raheem Sterling,0.6592,0.3032,Chelsea,MID,9.9,6.0,32.6,6.2,333.0,81.0,224895.0,24.0,Raheem Sterling,1.837904,0.496933,0.96,0.632832,0.291072,6.45
152,Harry Kane,0.7080,0.1760,Spurs,FWD,11.4,6.5,54.9,17.6,360.0,90.0,420033.0,26.0,Harry Kane,1.267004,0.376318,1.34,0.948720,0.235840,6.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,Lukasz Fabianski,0.0000,0.0000,West Ham,GK,5.0,3.2,7.0,1.0,298.0,90.0,15511.0,13.0,Lukasz Fabianski,0.703393,0.261056,0.84,0.000000,0.000000,0.70
127,Illan Meslier,0.0000,0.0000,Leeds,GK,4.5,3.5,10.1,2.5,360.0,90.0,59173.0,14.0,Illan Meslier,0.675292,0.255368,0.85,0.000000,0.000000,0.68
199,Danny Ward,0.0000,0.0000,Leicester,GK,4.0,1.0,5.5,27.0,360.0,90.0,501847.0,4.0,Danny Ward,0.435421,0.206792,0.94,0.000000,0.000000,0.44
44,Vicente Guaita,0.0000,0.0000,Crystal Palace,GK,4.5,1.5,8.1,1.4,360.0,90.0,22917.0,6.0,Vicente Guaita,0.370986,0.193682,0.92,0.000000,0.000000,0.37


In [217]:
df_final[df_final['team']=='Man City']

,name,npxg,xa,team,position,now_cost,form,ict_index,selected_by_percent,minutes,...,transfers_in,total_points,name_alt,ev_def,ev_mid,xgc_boost_x,xg_true,xa_true,xgc_boost_y,proj_score
125,Joao Cancelo,0.0000,0.0616,Man City,DEF,7.1,4.8,15.9,45.6,330.0,...,740988.0,19.0,João Cancelo,2.744932,0.698794,1.64,0.000000,0.101024,1.64,5.15
126,Ederson,0.0000,0.0000,Man City,GK,5.5,3.8,4.8,18.0,360.0,...,448135.0,15.0,Ederson Moraes,2.744932,0.698794,1.64,0.000000,0.000000,1.64,2.74
127,Kevin De Bruyne,0.1384,0.4224,Man City,MID,12.2,6.8,50.4,26.2,355.0,...,1112183.0,27.0,Kevin Bruyne,2.744932,0.698794,1.64,0.226976,0.692736,1.64,5.84
128,Kyle Walker,0.0000,0.0400,Man City,DEF,5.1,3.5,8.7,12.7,351.0,...,643024.0,14.0,Kyle Walker,2.744932,0.698794,1.64,0.000000,0.065600,1.64,5.01
129,Erling Haaland,1.0848,0.0672,Man City,FWD,11.8,10.2,51.4,66.7,323.0,...,2202845.0,41.0,Erling Haaland,2.744932,0.698794,1.64,1.779072,0.110208,1.64,7.57
130,Ilkay Gundogan,0.3040,0.1800,Man City,MID,7.6,6.2,37.3,5.2,260.0,...,440913.0,25.0,Ilkay Gündogan,2.744932,0.698794,1.64,0.498560,0.295200,1.64,6.04
131,Phil Foden,0.2160,0.2376,Man City,MID,8.0,4.5,36.5,7.4,312.0,...,352812.0,18.0,Phil Foden,2.744932,0.698794,1.64,0.354240,0.389664,1.64,5.62
132,Ruben Dias,0.0240,0.0000,Man City,DEF,6.0,3.5,14.6,10.9,312.0,...,214863.0,14.0,Rúben Dias,2.744932,0.698794,1.64,0.039360,0.000000,1.64,4.98
133,Bernardo Silva,0.4400,0.3800,Man City,MID,6.8,6.0,30.5,7.2,217.0,...,334279.0,24.0,Bernardo Silva,2.744932,0.698794,1.64,0.721600,0.623200,1.64,8.00
134,John Stones,0.0400,0.0000,Man City,DEF,5.4,1.0,6.1,2.0,206.0,...,21944.0,4.0,John Stones,2.744932,0.698794,1.64,0.065600,0.000000,1.64,5.14


In [261]:
df_dream_team = optimizer()

Assembling Dream Team...


In [262]:
df_dream_team

,name,club,position,price,proj_points
0,Mohamed Salah,Liverpool,MID,13.0,7.67
1,Harvey Elliott,Liverpool,MID,5.0,5.03
2,Harry Kane,Spurs,FWD,11.4,6.23
3,Ederson,Man City,GK,5.5,3.04
4,Erling Haaland,Man City,FWD,11.8,6.75
5,Bernardo Silva,Man City,MID,6.8,6.78
6,Harry Toffolo,Nott'm Forest,DEF,4.5,4.38
7,Neco Williams,Nott'm Forest,DEF,4.1,5.10
8,Reece James,Chelsea,DEF,6.1,5.81
9,Raheem Sterling,Chelsea,MID,9.9,6.45


In [216]:
display_results()


                name       club position  price  proj_points
0    Bernardo Silva   Man City      MID    6.8         8.00
1      Pascal Gross   Brighton      MID    5.8         6.51
2   Raheem Sterling    Chelsea      MID    9.9         6.17
3    Ilkay Gundogan   Man City      MID    7.6         6.04
4           Rodrigo      Leeds      MID    6.5         6.03
5           Jose Sa     Wolves       GK    5.0         2.73
6    Erling Haaland   Man City      FWD   11.8         7.57
7        Harry Kane      Spurs      FWD   11.4         5.64
8        Ivan Toney  Brentford      FWD    7.2         4.66
9    William Saliba    Arsenal      DEF    4.7         5.48
10      Reece James    Chelsea      DEF    6.1         5.32
11          Emerson      Spurs      DEF    5.0         5.13
12       Max Kilman     Wolves      DEF    4.5         4.91
13       Ben Davies      Spurs      DEF    5.0         4.75

                name       club position  price  proj_points
0    Bernardo Silva   Man City      

In [39]:
def optimizer(BUDGET=100):

    df = df_ev
    print('Assembling Dream Team...')
    # helper variables
    POS = set(df['position'])
    CLUBS = set(df['team'])
    pos_available = {
        'GK': 1,
        'DEF': 5,
        'MID': 5,
        'FWD': 3,
    }
    players = [LpVariable(f'player_{i}', cat='Binary') for i in df.index]

    # initialize variables
    names = list(df['name'])
    teams = list(df['team'])
    positions = list(df['position'])
    prices = list(df['now_cost'])
    points = list(df['proj_score'])
    # difficulty = list(df['fixture_difficulty'])

    # problem
    prob = LpProblem('FPL_Dream_Team', LpMaximize)

    # objective function
    prob += lpSum(players[i] * points[i] for i in range(len(df)))

    # constraints
    prob += lpSum(players[i] * df['now_cost'][df.index[i]] for i in range(len(df))) <= BUDGET # Budget Limit

    for pos in POS:
        prob += lpSum(players[i] for i in range(len(df)) if positions[i] == pos) <= pos_available[pos] # Position Limit

    for club in CLUBS:
        prob += lpSum(players[i] for i in range(len(df)) if teams[i] == club) <= 3 # Club Limit

    prob.solve()

    tup_list = []
    for v in prob.variables():
        if v.varValue != 0:
            ind = int(v.name.split("_")[1])

            name = df['name'][ind]
            club = df['team'][ind]
            position = df['position'][ind]
            point = df['proj_score'][ind]
            price = df['now_cost'][ind]
            # difficulty = df['fixture_difficulty'][ind]
            
            tup = (name, club, position, price, point)
            tup_list.append(tup)

    df_dream_team = pd.DataFrame(tup_list, columns=['name', 'club', 'position', 'price', 'proj_points'])
    return df_dream_team

In [215]:
def display_results():

    df_dream_team.sort_values(['position', 'proj_points'], ascending=False, inplace=True, ignore_index=True)
    print('\n', df_dream_team)
    print('\n', df_dream_team.sort_values(['proj_points', 'price'], ascending=False, ignore_index=True))
    print(f"\nTeam Value: {round(df_dream_team['price'].sum(), 2)}")
    #print(f"\nAverage Difficulty: {round(df_dream_team['difficulty'].mean(), 2)}")
    print(f"\nProjected Points: {round(df_dream_team['proj_points'].sum(), 0)}\n")

    col_to_keep = ['name', 'team', 'position', 'now_cost', 'proj_score']
    print(df_final.sort_values('proj_score').groupby('position').tail(8).sort_values(['position', 'proj_score'], ascending=False)[col_to_keep])


In [26]:
df_3wk_1 = df_3wk[1][['name', 'proj_score']]
df_3wk_2 = df_3wk[2][['name', 'proj_score']]

In [32]:
df_3wk_merged = df_3wk[0].merge(df_3wk_1, on='name', how='left').merge(df_3wk_2, on='name', how='left')
df_3wk_merged = df_3wk_merged.rename(columns={'proj_score_x': 'proj_score', 'proj_score_y': 'proj_score_1', 'proj_score': 'proj_score_2'})

,name,npxg,xa,team,position,now_cost,form,ict_index,selected_by_percent,minutes,...,total_points,name_alt,ev_def,ev_mid,xgc_boost,xg_true,xa_true,proj_score,proj_score_1,proj_score_2
0,Lucas Digne,0.02880,0.08976,Aston Villa,DEF,4.7,0.8,23.0,2.1,450.0,...,4.0,Lucas Digne,-0.278176,0.033189,0.39,0.011232,0.035006,1.93,3.78,3.53
1,Matty Cash,0.02400,0.06880,Aston Villa,DEF,4.7,1.2,15.8,7.0,450.0,...,6.0,Matty Cash,-0.278176,0.033189,0.39,0.009360,0.026832,1.89,3.68,3.42
2,Boubacar Kamara,0.00000,0.00000,Aston Villa,MID,4.9,1.8,11.7,0.2,433.0,...,9.0,Boubacar Kamara,-0.278176,0.033189,0.39,0.000000,0.000000,2.03,2.38,2.33
3,John McGinn,0.01296,0.01728,Aston Villa,MID,5.3,1.6,13.5,0.9,414.0,...,8.0,John McGinn,-0.278176,0.033189,0.39,0.005054,0.006739,2.08,2.49,2.44
4,Jacob Ramsey,0.02160,0.10160,Aston Villa,MID,5.2,1.2,16.9,0.9,339.0,...,6.0,Jacob Ramsey,-0.278176,0.033189,0.39,0.008424,0.039624,2.19,2.77,2.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,Raheem Sterling,0.59552,0.22192,Chelsea,MID,10.0,6.4,45.3,9.1,423.0,...,32.0,Raheem Sterling,1.812416,0.491448,0.97,0.577654,0.215262,5.96,7.03,5.13
168,Kai Havertz,0.21184,0.01440,Chelsea,FWD,7.8,1.6,19.4,2.5,408.0,...,8.0,Kai Havertz,1.812416,0.491448,0.97,0.205485,0.013968,2.86,3.20,2.73
169,Mason Mount,0.09904,0.15744,Chelsea,MID,7.7,2.6,35.3,4.4,365.0,...,13.0,Mason Mount,1.812416,0.491448,0.97,0.096069,0.152717,3.43,3.61,2.97
170,Jorginho,0.00000,0.00864,Chelsea,MID,6.0,3.8,16.0,3.5,365.0,...,19.0,Jorge Filho,1.812416,0.491448,0.97,0.000000,0.008381,2.52,2.34,2.20


In [51]:
df_3wk_merged[df_3wk_merged['team']=='Liverpool']

,name,npxg,xa,team,position,now_cost,form,ict_index,selected_by_percent,minutes,...,name_alt,ev_def,ev_mid,xgc_boost,xg_true,xa_true,proj_score,proj_score_1,proj_score_2,mean_score
65,Trent Alexander Arnold,0.05728,0.09696,Liverpool,DEF,7.5,4.6,40.4,48.9,422.0,...,Trent Alexander-Arnold,1.383192,0.400501,1.34,0.076755,0.129926,4.36,3.98,3.62,3.986667
66,Alisson,0.00000,0.00000,Liverpool,GK,5.5,2.4,7.4,15.4,450.0,...,Alisson Becker,1.383192,0.400501,1.34,0.000000,0.000000,1.38,1.38,0.99,1.250000
67,Mohamed Salah,0.42288,0.30208,Liverpool,MID,13.0,7.0,61.7,52.8,450.0,...,Mohamed Salah,1.383192,0.400501,1.34,0.566659,0.404787,6.38,4.84,4.88,5.366667
68,Virgil van Dijk,0.06400,0.00000,Liverpool,DEF,6.5,3.6,20.7,14.1,450.0,...,Virgil Dijk,1.383192,0.400501,1.34,0.085760,0.000000,3.90,3.69,3.32,3.636667
69,Luis Diaz,0.41456,0.00000,Liverpool,MID,8.2,5.8,42.7,30.2,437.0,...,Luis Díaz,1.383192,0.400501,1.34,0.555510,0.000000,5.13,4.07,4.07,4.423333
70,Andrew Robertson,0.03264,0.06880,Liverpool,DEF,6.8,3.0,23.7,7.4,374.0,...,Andrew Robertson,1.383192,0.400501,1.34,0.043738,0.092192,4.01,3.76,3.40,3.723333
71,Harvey Elliott,0.11200,0.28560,Liverpool,MID,5.0,3.0,32.1,1.2,342.0,...,Harvey Elliott,1.383192,0.400501,1.34,0.150080,0.382704,4.29,3.55,3.52,3.786667
72,Jordan Henderson,0.03600,0.08320,Liverpool,MID,5.4,1.8,15.4,0.8,314.0,...,Jordan Henderson,1.383192,0.400501,1.34,0.048240,0.111488,2.98,2.75,2.69,2.806667
73,Roberto Firmino,0.48000,0.22240,Liverpool,FWD,8.0,6.8,53.0,4.0,298.0,...,Roberto Firmino,1.383192,0.400501,1.34,0.643200,0.298016,5.39,4.08,4.18,4.550000
74,Joe Gomez,0.04000,0.00000,Liverpool,DEF,4.5,2.0,7.7,0.6,297.0,...,Joseph Gomez,1.383192,0.400501,1.34,0.053600,0.000000,3.70,3.58,3.20,3.493333


In [53]:
df_3wk_merged['mean_score'] = df_3wk_merged[['proj_score', 'proj_score_1', 'proj_score_2']].sum(axis=1) / 3
df_3wk_merged.sort_values('mean_score', ascending=False).head(20)

,name,npxg,xa,team,position,now_cost,form,ict_index,selected_by_percent,minutes,...,name_alt,ev_def,ev_mid,xgc_boost,xg_true,xa_true,proj_score,proj_score_1,proj_score_2,mean_score
133,Erling Haaland,1.49088,0.04032,Man City,FWD,11.9,11.6,70.3,74.6,391.0,...,Erling Haaland,3.035453,0.766255,1.24,1.848691,0.049997,7.46,5.93,6.34,6.576667
167,Raheem Sterling,0.59552,0.22192,Chelsea,MID,10.0,6.4,45.3,9.1,423.0,...,Raheem Sterling,1.812416,0.491448,0.97,0.577654,0.215262,5.96,7.03,5.13,6.040000
138,John Stones,0.02400,0.28000,Man City,DEF,5.4,2.8,10.7,2.2,280.0,...,John Stones,3.035453,0.766255,1.24,0.029760,0.347200,6.60,5.03,5.53,5.720000
21,Gabriel Jesus,0.70832,0.15904,Arsenal,FWD,8.2,6.4,64.6,80.1,430.0,...,Gabriel Jesus,2.044314,0.541723,1.00,0.708320,0.159040,5.21,6.16,4.79,5.386667
67,Mohamed Salah,0.42288,0.30208,Liverpool,MID,13.0,7.0,61.7,52.8,450.0,...,Mohamed Salah,1.383192,0.400501,1.34,0.566659,0.404787,6.38,4.84,4.88,5.366667
52,Aleksandar Mitrovic,0.67584,0.08000,Fulham,FWD,6.8,6.8,57.6,23.0,450.0,...,Aleksandar Mitrović,0.095179,0.136396,0.71,0.479846,0.056800,4.07,4.45,7.10,5.206667
18,William Saliba,0.01440,0.23200,Arsenal,DEF,4.8,5.8,16.5,19.9,450.0,...,William Saliba,2.044314,0.541723,1.00,0.014400,0.232000,5.06,5.62,4.70,5.126667
82,Jack Harrison,0.19200,0.38816,Leeds,MID,6.1,5.6,46.6,5.6,418.0,...,Jack Harrison,0.675292,0.255368,0.86,0.165120,0.333818,4.08,6.71,4.42,5.070000
32,Pascal Gross,0.30688,0.19728,Brighton,MID,5.9,7.4,44.9,22.1,450.0,...,Pascal Groß,1.728711,0.473502,0.94,0.288467,0.185443,4.47,5.49,4.79,4.916667
23,Ben White,0.00000,0.17296,Arsenal,DEF,4.5,4.4,11.4,5.5,392.0,...,Benjamin White,2.044314,0.541723,1.00,0.000000,0.172960,4.74,5.19,4.42,4.783333


In [12]:
# df_player_xg = fbref_parser()
df_fpl = fpl_parser()

df_3wk = []
week=6
for i in range(week, week+3):
    team_xg_df, team_xgc_df = team_xg(week=i)
    df_xg, df_xgc = fixture_xg(week=i)
    df_xgc = xgc_parser()
    df_final = merge_df()
    df_ev = compute_ev()
    df_3wk.append(df_ev)
# df_dream_team = optimizer()

0/359 completed.
50/359 completed.
100/359 completed.
150/359 completed.
200/359 completed.
250/359 completed.
300/359 completed.
350/359 completed.
Parsing FPL data...
Skipping player 611 due to error encountered
Skipping player 613 due to error encountered
Skipping player 614 due to error encountered
Skipping player 612 due to error encountered
Skipping player 610 due to error encountered
Skipping player 609 due to error encountered
Skipping player 608 due to error encountered
Dropping these rows due to missing data...
Empty DataFrame
Columns: [name, npxg, xa, team, position, now_cost, form, ict_index, selected_by_percent, minutes, mins_ma, transfers_in, total_points, name_alt, ev_def, ev_mid, xgc_boost, xg_true, xa_true, proj_score]
Index: []
Parsing FPL data...
Skipping player 611 due to error encountered
Skipping player 613 due to error encountered
Skipping player 614 due to error encountered
Skipping player 612 due to error encountered
Skipping player 610 due to error encountered

In [10]:
# # main
# df_player_xg = fbref_parser()
# df_fpl = fpl_parser()
# df_xg = xg_parser()
# dgw_list, bgw_list = calc_gw()
# df_final = merge_df()
# df_final = compute_ev()
# df_dream_team = optimizer(BUDGET=100)

0/675 completed.
20/675 completed.
40/675 completed.
60/675 completed.
80/675 completed.
100/675 completed.
120/675 completed.
140/675 completed.
160/675 completed.
HTTPSConnectionPool(host='fbref.com', port=443): Max retries exceeded with url: /en/players/1c7012b8/matchlogs/2021-2022/summary/Declan-Rice-Match-Logs (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x1075b9400>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
https://fbref.com/en/players/1c7012b8/Declan-Rice
HTTPSConnectionPool(host='fbref.com', port=443): Max retries exceeded with url: /en/players/6613c819/matchlogs/2021-2022/summary/Tomas-Soucek-Match-Logs (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x1075b9630>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
https://fbref.com/en/players/6613c819/Tomas-Soucek
180/675 completed.
200/675 completed.
220/675 completed